In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf #tensorflow
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load
%matplotlib inline
from matplotlib import pyplot as plt

print(tf.__version__)

1.0.1


In [4]:
DATA_PATH = 'cifar10/cifar-10-batches-py/'
CHECKPOINT = 'simple_res_net_0525.ckpt'
SAVE_PATH = 'checkpoint/simple_res_net/'
BOARD_PATH = './tensorboard/board_res_net'

In [31]:
#npz - python numpy 파일 포맷. > dictionary로 불러옴 (Save several arrays into a single file in uncompressed .npz format)
def load_npz(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        return dict
    
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data']) #데이터 갯수
    start_idx = list(range(0, num_images, batch_size)) #각 배치별 시작 인덱스 리스트
    end_idx = list(range(batch_size, num_images + 1, batch_size))#각 배치별 끝 인덱스 
    if allow_small_batch and end_idx[-1] < num_images : #스몰배치 허용시 마지막 배치
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

#one hot encoding
def one_hot_encode(x, numclass):
    return np.eye(numclass)[x] #numclass 크기의 항등행렬 생성 

#############################################################################################################
######################################아래는 방식변환중
#############################################################################################################

#CIFAR 데이터 읽어오기 
def read_dataset(filename_queue):
    class DataRecord(object):
        pass
    
    result = DataRecord()
    
    label_bytes = 1
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    result.key, value = reader.read(filename_queue)
    print("key & value")
    print(result.key)
    print (value)
    record_bytes = tf.decode_raw(value, tf.uint8)
    
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32) #input, begin, end로 잘라냄 > 0~1
    print(result.label)
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], # 1부터 image_bytes까지 image를 잘라낸 후 
                                              [label_bytes + image_bytes]), #channel x height x width로 변환함.
                             [result.depth, result.height, result.width])
    
    result.uint8image = tf.transpose(depth_major, [1,2,0]) #channel x height x width >>> height x width x chennl로 변경
    
    return result

#label, image 세팅해줌 
def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
        in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      shuffle: boolean indicating whether to use a shuffling queue.
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16 #프로세스 스레드 
    if shuffle: #셔플 하면 
        images, label_batch = tf.train.shuffle_batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples) #이거만 해주면 batch와 동일함. 
    #Minimum number elements in the queue after a dequeue, used to ensure a level of mixing of elements.
    else:
        images, label_batch = tf.train.batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images) #걍 보드용인듯
    
    print("label_batch")
    print(label_batch)
    return images, tf.reshape(label_batch, [batch_size])


def distorted_inputs(data_dir, batch_size): #좀 드럽게 인풋받아오기 
    """Construct distorted input for CIFAR training using the Reader ops.
    Args:
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    filenames = [os.path.join(data_dir, 'data_batch_%d' % i) for i in range(1, 6)] 
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_dataset(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = 32
    width = 32

    # Image processing for training the network.  Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])

    # Randomly flip the image horizontally.
    distorted_image = tf.image.random_flip_left_right(distorted_image)

    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    distorted_image = tf.image.random_brightness(distorted_image,
                                               max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image,
                                             lower=0.2, upper=1.8)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(distorted_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(10000 * min_fraction_of_examples_in_queue)
    print('Filling queue with %d CIFAR images before starting to train. '
         'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=True)

print("FUNCTIONS READY")

FUNCTIONS READY


In [32]:
#NETWORK PARAMETERS

stddev = 0.1

weights = {
     'conv' : tf.Variable(tf.random_normal([3, 3, 3, 16], stddev=stddev), name='conv'),
    'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev, name='conv1_1x1')),
    'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev, name='conv1_3x3')),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev, name='conv1_1x1_16')),
    'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev, name='conv2_1x1')),
    'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev, name='conv2_3x3')),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev, name='conv2_1x1_16')),
    
    # conv 16 + conv2 16 = 32filters 
    
    'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev, name='conv3_1x1')),
    'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev, name='conv3_3x3')),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev, name='conv3_1x1_32')),
    'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev, name='conv4_1x1')),
    'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev, name='conv4_3x3')),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev, name='conv4_1x1_32')),
    
    # conv2 지난거32 + conv4 32 = 64
    
    'conv5_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev, name='conv5_1x1')),
    'conv5_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev, name='conv5_3x3')),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev, name='conv5_1x1_64')),
    'conv6_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev, name='conv6_1x1')),
    'conv6_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev, name='conv6_3x3')),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev, name='conv6_1x1_64')),
    
    #conv4 지난거 64 + conv6 64 = 128
    
    'dense1' : tf.Variable(tf.random_normal([16*16*128, 1000], stddev=stddev, name='dense1')),
    'dense2' : tf.Variable(tf.random_normal([1000, 10], stddev=stddev, name='dense2'))    
}
biases = {
    'conv' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv_b')),
    'conv1_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_1x1_b')),
    'conv1_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_3x3_b')),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv1_1x1_16_b')),
    'conv2_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_1x1_b')),
    'conv2_3x3' :tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_3x3_b')),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv2_1x1_16_b')),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_1x1_b')),
    'conv3_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_3x3_b')),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv3_1x1_32_b')),
    'conv4_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_1x1_b')),
    'conv4_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_3x3_b')),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv4_1x1_32_b')),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_1x1_b')),
    'conv5_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_3x3_b')),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv5_1x1_64_b')),
    'conv6_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_1x1_b')),
    'conv6_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_3x3_b')),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv6_1x1_64_b')),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev, name='dense1_b')),
    'dense2' : tf.Variable(tf.random_normal([10], stddev=stddev, name='dense2_b'))
}

In [33]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='ResNet'):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        with tf.name_scope('conv') as scope:
            conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, _b['conv'])
            conv = tf.nn.relu(conv)
            network['conv'] = conv
        
        #16
        with tf.name_scope('conv2') as scope:
            conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
            conv1_1x1 = tf.nn.relu(conv1_1x1)
            conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
            conv1_3x3 = tf.nn.relu(conv1_3x3)
            conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
            conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
            network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        with tf.name_scope('conv3') as scope:
            conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
            conv2_1x1 = tf.nn.relu(conv2_1x1)
            conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
            conv2_3x3 = tf.nn.relu(conv2_3x3)
            conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
            #print(conv2_1x1_16.shape)
            #32
            conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
            conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
            network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        with tf.name_scope('conv4') as scope:
            conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
            conv3_1x1 = tf.nn.relu(conv3_1x1)
            conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
            conv3_3x3 = tf.nn.relu(conv3_3x3)
            conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
            conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
            network['conv3_1x1_32'] = conv3_1x1_32
        
        with tf.name_scope('conv5') as scope:
            conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
            conv4_1x1 = tf.nn.relu(conv4_1x1)
            conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
            conv4_3x3 = tf.nn.relu(conv4_3x3)
            conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])

            #64
            conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
            conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
            network['conv4_1x1_32'] = conv4_1x1_32
    
        with tf.name_scope('conv6') as scope:
            conv5_1x1 = tf.nn.conv2d(conv4_1x1_32, _w['conv5_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_1x1 = tf.nn.bias_add(conv5_1x1, _b['conv5_1x1'])
            conv5_1x1 = tf.nn.relu(conv5_1x1)
            conv5_3x3 = tf.nn.conv2d(conv5_1x1, _w['conv5_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_3x3 = tf.nn.bias_add(conv5_3x3, _b['conv5_3x3'])
            conv5_3x3 = tf.nn.relu(conv5_3x3)
            conv5_1x1_64 = tf.nn.conv2d(conv5_3x3, _w['conv5_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_1x1_64 = tf.nn.bias_add(conv5_1x1_64, _b['conv5_1x1_64'])
            conv5_1x1_64 = tf.nn.relu(conv5_1x1_64)
            network['conv5_1x1_64'] = conv5_1x1_64
        
        with tf.name_scope('conv7') as scope:
            conv6_1x1 = tf.nn.conv2d(conv5_1x1_64, _w['conv6_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_1x1 = tf.nn.bias_add(conv6_1x1, _b['conv6_1x1'])
            conv6_1x1 = tf.nn.relu(conv6_1x1)
            conv6_3x3 = tf.nn.conv2d(conv6_1x1, _w['conv6_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_3x3 = tf.nn.bias_add(conv6_3x3, _b['conv6_3x3'])
            conv6_3x3 = tf.nn.relu(conv6_3x3)
            conv6_1x1_64 = tf.nn.conv2d(conv6_3x3, _w['conv6_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_1x1_64 = tf.nn.bias_add(conv6_1x1_64, _b['conv6_1x1_64'])

            #128
            conv6_1x1_64 = tf.concat([conv4_1x1_32, conv6_1x1_64],3)
            conv6_1x1_64 = tf.nn.relu(conv6_1x1_64)
            network['conv6_1x1_64'] = conv6_1x1_64
        
        with tf.name_scope('pool') as scope:
            pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            network['pool'] = pool
        
        with tf.name_scope('dense1') as scope:
            dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
            dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
            dense1 = tf.nn.relu(dense1)
            network['dense1'] = dense1
            
        with tf.name_scope('logit') as scope:
            logit = tf.add(tf.matmul(dense1, _w['dense2']), _b['dense2'])
            network['logit'] = logit
        
    return network

In [34]:
images, labels = distorted_inputs(DATA_PATH, 64)
print(images)
print(images.shape)
labels = tf.cast(labels, tf.int64)
print(labels.shape)

out = ResNet(32, 32, 3, images, weights, biases, 'ResNet')
for key, value in out.items():
    print (key, value)


key & value
Tensor("ReaderReadV2_9:0", shape=(), dtype=string)
Tensor("ReaderReadV2_9:1", shape=(), dtype=string)
Tensor("Cast_30:0", dtype=int32)
Filling queue with 4000 CIFAR images before starting to train. This will take a few minutes.
label_batch
Tensor("shuffle_batch_6:1", shape=(64, 1), dtype=int32)
Tensor("shuffle_batch_6:0", shape=(64, 32, 32, 3), dtype=float32)
(64, 32, 32, 3)
(64,)
conv Tensor("ResNet_3/conv/Relu:0", shape=(64, 32, 32, 16), dtype=float32)
conv1_1x1_16 Tensor("ResNet_3/conv2/Relu_2:0", shape=(64, 32, 32, 16), dtype=float32)
conv2_1x1_16 Tensor("ResNet_3/conv3/Relu_2:0", shape=(64, 32, 32, 32), dtype=float32)
conv3_1x1_32 Tensor("ResNet_3/conv4/Relu_2:0", shape=(64, 32, 32, 32), dtype=float32)
conv4_1x1_32 Tensor("ResNet_3/conv5/Relu_2:0", shape=(64, 32, 32, 64), dtype=float32)
conv5_1x1_64 Tensor("ResNet_3/conv6/Relu_2:0", shape=(64, 32, 32, 64), dtype=float32)
conv6_1x1_64 Tensor("ResNet_3/conv7/Relu_2:0", shape=(64, 32, 32, 128), dtype=float32)
pool Tensor(

In [35]:
images, labels = distorted_inputs(DATA_PATH, 64)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=out['logit']))
opt = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(labels,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

key & value
Tensor("ReaderReadV2_10:0", shape=(), dtype=string)
Tensor("ReaderReadV2_10:1", shape=(), dtype=string)
Tensor("Cast_35:0", dtype=int32)
Filling queue with 4000 CIFAR images before starting to train. This will take a few minutes.
label_batch
Tensor("shuffle_batch_7:1", shape=(64, 1), dtype=int32)


In [14]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    coord = tf.train.Coordinator()
    thresds = tf.train.start_queue_runners(coord=coord)
    
    for i in range(3):
        la, label = sess.run([images, labels])
        print(la.shape)
        print(label.shape)
        print(label)
        
        #_ = sess.run(opt)
        #if i % 10 == 0:
           # print(str(i) + " > loss : " + str(loss) + ", accr : " + str(accr))
    
    coord.request_stop()
    coord.join(thresds)

(64, 32, 32, 3)
(64,)
[178  80  44  29 113 136 102  95 213  98 136  31 130 255  76  58 179 151
  67  51 105 165  92  23 133  84  45 236  89  40 246 198  70 237 253 100
  45 185 241 129  31  80  11 143  95 145 226 252 203 132 201 254 181  15
 255 108 143  87 255  65 124  21 192  55]
(64, 32, 32, 3)
(64,)
[212 156  64 148  78  79 111 163 120 120 196  72  61 132  48 156  78  36
  48 180 103  58 119 174 166 119  76 184 131 208  54 102 207 103 152  54
 165 103  78 129 119  76  58  65 113  76  41  85  53  58 207 181  63 139
 170  53 230 127 239 108  20  23 155 109]
(64, 32, 32, 3)
(64,)
[  4  82  91  51 199 109 113 166 252 103 231 116  76 171  12 211 132  14
 112  18  87 111 103 198 132 121  57  12 174   3 198 191 140 217  80  89
  92 165  60 140 138 145 201 156  87  57  29 132 113  92 155 119 113  38
  34 145  41 127  89 228  56 213 181 145]


In [36]:
#input & output 
x = tf.placeholder("float", [64, 32*32 * 3]) #batch x image size
y = tf.placeholder("float", [63, 10]) #batch x class`

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']
LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [38]:
file_name = DATA_PATH + train_file[0] #해당 파일 이름
print(file_name)
load_data = load_npz(file_name) #해당 파일 이름으로 불러온다. 

cifar10/cifar-10-batches-py/data_batch_1


AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    coord = tf.train.Coordinator()
    thresds = tf.train.start_queue_runners(coord=coord)
    
    for i in range(5):
        la = sess.run(images)
        print(la)
        
        #_ = sess.run(opt)
        #if i % 10 == 0:
           # print(str(i) + " > loss : " + str(loss) + ", accr : " + str(accr))
    
    coord.request_stop()
    coord.join(thresds)